In [7]:
import os
import shutil
import csv
import re

In [2]:
pwd

'/Users/cpan/Desktop'

In [3]:
# !pip install pathlib

In [9]:
import os
import re
import subprocess
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

def find_matching_files(source_dir, text_file):
    with open(text_file, 'r') as f:
        medical_record_numbers = f.read().splitlines()

    # Only search in relevant subdirectories (customize this if you know the structure)
    relevant_subdirs = ['subdir1', 'subdir2', 'subdir3']
    all_files = [file for subdir in relevant_subdirs
                 for file in Path(source_dir, subdir).rglob('*') if file.is_file()]

    # Compile the regular expression pattern for faster matching
    # compiled_patterns = [re.compile(rf"^{number}(_\d+)?\.pdf$") for number in medical_record_numbers]
    compiled_patterns = [re.compile(rf"^{number}(_\d+)?\.pdf(\*)?$", re.IGNORECASE) for number in medical_record_numbers]

    matching_files = []
    with ThreadPoolExecutor() as executor:
        for file in all_files:
            for pattern in compiled_patterns:
                if pattern.match(file.name):
                    matching_files.append(file)
                    break  # No need to check other patterns for this file

    return matching_files

def calculate_total_size_with_du(files):
    cmd = ['du', '-ch'] + [str(file) for file in files]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, text=True)
    total_size_output = result.stdout.strip().split('\n')[-1]

    # Extract the total size and unit from the output using regex
    match = re.search(r'(\d+\.\d+|\d+)\s*([A-Za-z]+)', total_size_output)
    if match:
        total_size_str, unit = match.groups()
        unit = unit.upper()

        # Convert the total size to bytes based on the unit
        total_size_bytes = float(total_size_str) * (1024 ** {'B': 0, 'K': 1, 'M': 2, 'G': 3}[unit])
        return int(total_size_bytes)

    return 0

# Specify the paths to the source directory and the text file
source_dir = Path("/Volumes/Billing/Medical Records")
text_file = "/Users/cpan/Desktop/project/source_data/medical_records.txt"

# Find matching files
matching_files = find_matching_files(source_dir, text_file)

# Calculate total file size using du command
total_size_bytes = calculate_total_size_with_du(matching_files)
total_size_gb = total_size_bytes / (1024 ** 3)

print(f"Total size of matching files: {total_size_gb:.2f} GB")


Total size of matching files: 147.00 GB


In [1]:
# delete duplicate files in the copied directory 
import os
from pathlib import Path

destination_dir = Path("/Users/cpan/Desktop/all_medical_pdfs")

# Function to get the patient number from the file name
def get_patient_number(file_name):
    return file_name.split("_")[0]

# Create a dictionary to store files grouped by patient number
patient_files = {}

# Iterate through all files in the destination directory
for file in destination_dir.glob("*.pdf"):
    patient_number = get_patient_number(file.stem)

    # Add the file to the patient_files dictionary
    if patient_number in patient_files:
        patient_files[patient_number].append(file)
    else:
        patient_files[patient_number] = [file]

# Delete duplicates and keep only the oldest file for each patient number
for patient_number, files in patient_files.items():
    if len(files) > 1:
        # Sort the files by their modified time (oldest first)
        files.sort(key=lambda x: os.path.getmtime(x))

        # Delete all files except the oldest one
        for file in files[1:]:
            os.remove(file)
            print(f"Deleted duplicate file: {file.name}")

print("Duplicate files have been deleted.")

Deleted duplicate file: A39731_2.pdf
Deleted duplicate file: A42432_2.pdf
Deleted duplicate file: A42459_2.pdf
Deleted duplicate file: A36362_2.pdf
Deleted duplicate file: A43014_2.pdf
Deleted duplicate file: A43014_3.pdf
Deleted duplicate file: A42195_2.pdf
Deleted duplicate file: A42501_2.pdf
Deleted duplicate file: A40755_2.pdf
Deleted duplicate file: A42168_2.pdf
Deleted duplicate file: A31190_2.pdf
Deleted duplicate file: A31190_3.pdf
Deleted duplicate file: A42584_2.pdf
Deleted duplicate file: A28238_1.pdf
Deleted duplicate file: A29641_2.pdf
Deleted duplicate file: A42464_2.pdf
Deleted duplicate file: A28832_2.pdf
Deleted duplicate file: A35091_2.pdf
Deleted duplicate file: A40423_2.pdf
Deleted duplicate file: A29495_2.pdf
Deleted duplicate file: A42208_2.pdf
Deleted duplicate file: A41701_2.pdf
Deleted duplicate file: A43102_2.pdf
Deleted duplicate file: A41072_2.pdf
Deleted duplicate file: A39136_2.pdf
Deleted duplicate file: A42405_2.pdf
Deleted duplicate file: A42962_2.pdf
D

In [2]:
# rsync copy 
import os
import subprocess
from pathlib import Path
import time

source_dir = Path("/Volumes/Billing/Medical Records")
destination_dir = Path("/Users/cpan/Desktop/all_medical_pdfs")

text_file = "/Users/cpan/Desktop/project/source_data/medical_records.txt"

# Read the file names from the text file and clean them up
with open(text_file, "r") as file:
    text_file_names = [line.strip().replace("*", "") for line in file]

# Get all patient numbers in the destination directory
destination_patients = set()
for destination_file in destination_dir.glob("*.pdf"):
    destination_patients.add(destination_file.stem.split("_")[0])

# Copy the matching PDF files from the source directory to the destination directory
start_time = time.time()
pdf_count = 0

for i, file_name in enumerate(text_file_names, 1):
    source_file = source_dir / file_name
    destination_file = destination_dir / file_name

    # Extract the patient number from the file name
    patient_number = file_name.split("_")[0]

    # Check if the patient number already exists in the destination directory
    if patient_number in destination_patients:
        print(f"File {i}/{len(text_file_names)}: Skipped {destination_file.name} (Duplicate patient file)")
        continue

    # Check if the file already exists in the destination directory
    if destination_file.exists():
        print(f"File {i}/{len(text_file_names)}: Skipped {destination_file.name} (Already exists)")
        continue

    try:
        start_extraction_time = time.time()
        # Use rsync to copy the file
        subprocess.run(["rsync", "-av", str(source_file), str(destination_file)])
        end_extraction_time = time.time()
        extraction_time = end_extraction_time - start_extraction_time

        # Check if the file is a PDF
        if destination_file.suffix.lower() == ".pdf":
            pdf_count += 1

        print(f"File {i}/{len(text_file_names)}: Successfully copied {destination_file.name} (Extraction Time: {extraction_time:.2f} seconds)")

    except Exception as e:
        print(f"File {i}/{len(text_file_names)}: Error copying {destination_file.name}: {str(e)}")

end_time = time.time()
total_time = end_time - start_time

# Calculate the percentage of PDF files
percentage_pdf = (pdf_count / len(text_file_names)) * 100

print(f"\nTotal Execution Time: {total_time:.2f} seconds")
print(f"Percentage of PDF Files: {percentage_pdf:.2f}%")


File 1/103742: Skipped 39951654.pdf (Already exists)
File 2/103742: Skipped 7645D6FC.tmp (Already exists)
File 3/103742: Skipped A00001_MedRec1.pdf (Duplicate patient file)
File 4/103742: Skipped A00001_MedRec2.pdf (Duplicate patient file)
File 5/103742: Skipped A00001_MedRec3.pdf (Duplicate patient file)
File 6/103742: Skipped A00001_MedRec4.pdf (Duplicate patient file)
File 7/103742: Skipped A00001_MedRec5.pdf (Duplicate patient file)
File 8/103742: Skipped A00001_MedRec6.pdf (Duplicate patient file)
File 9/103742: Skipped A00001_MedRec7.pdf (Duplicate patient file)
File 10/103742: Skipped A00001_MedRec8.pdf (Duplicate patient file)
File 11/103742: Skipped A00852.pdf (Already exists)
File 12/103742: Skipped A00903.pdf (Already exists)
File 13/103742: Skipped A0100049.pdf (Already exists)
File 14/103742: Skipped A0100056.pdf (Already exists)
File 15/103742: Skipped A0100058.pdf (Already exists)
File 16/103742: Skipped A0100060.pdf (Already exists)
File 17/103742: Skipped A0100062.pdf 

File 3885/103742: Skipped A0115589.pdf (Already exists)
File 3886/103742: Skipped A0115593.pdf (Already exists)
File 3887/103742: Skipped A0115594.pdf (Already exists)
File 3888/103742: Skipped A0115596.pdf (Already exists)
File 3889/103742: Skipped A0115597.pdf (Already exists)
File 3890/103742: Skipped A0115599.pdf (Already exists)
File 3891/103742: Skipped A0115609.pdf (Already exists)
File 3892/103742: Skipped A0115610.pdf (Already exists)
File 3893/103742: Skipped A0115612.pdf (Already exists)
File 3894/103742: Skipped A0115620.pdf (Already exists)
File 3895/103742: Skipped A0115632.pdf (Already exists)
File 3896/103742: Skipped A0115633.pdf (Already exists)
File 3897/103742: Skipped A0115634.pdf (Already exists)
File 3898/103742: Skipped A0115634_2.pdf (Duplicate patient file)
File 3899/103742: Skipped A0115635.pdf (Already exists)
File 3900/103742: Skipped A0115639.pdf (Already exists)
File 3901/103742: Skipped A0115642.pdf (Already exists)
File 3902/103742: Skipped A0115646.pdf

File 8074/103742: Skipped A0140125.pdf (Already exists)
File 8075/103742: Skipped A0140129.pdf (Already exists)
File 8076/103742: Skipped A0140132.pdf (Already exists)
File 8077/103742: Skipped A0140138.pdf (Already exists)
File 8078/103742: Skipped A0140231.pdf (Already exists)
File 8079/103742: Skipped A0140234.pdf (Already exists)
File 8080/103742: Skipped A0140235.pdf (Already exists)
File 8081/103742: Skipped A0140243.pdf (Already exists)
File 8082/103742: Skipped A0140246.pdf (Already exists)
File 8083/103742: Skipped A0140248.pdf (Already exists)
File 8084/103742: Skipped A0140254.pdf (Already exists)
File 8085/103742: Skipped A0140260.pdf (Already exists)
File 8086/103742: Skipped A0140263.pdf (Already exists)
File 8087/103742: Skipped A0140265.pdf (Already exists)
File 8088/103742: Skipped A0140265_2.pdf (Duplicate patient file)
File 8089/103742: Skipped A0140266.pdf (Already exists)
File 8090/103742: Skipped A0140270.pdf (Already exists)
File 8091/103742: Skipped A0140271.pdf

File 11977/103742: Skipped A0159982.pdf (Already exists)
File 11978/103742: Skipped A0159986.pdf (Already exists)
File 11979/103742: Skipped A0159992.pdf (Already exists)
File 11980/103742: Skipped A0159994.pdf (Already exists)
File 11981/103742: Skipped A0159996.pdf (Already exists)
File 11982/103742: Skipped A0160008.pdf (Already exists)
File 11983/103742: Skipped A0160010.pdf (Already exists)
File 11984/103742: Skipped A0160020.pdf (Already exists)
File 11985/103742: Skipped A0160027.pdf (Already exists)
File 11986/103742: Skipped A0160030.pdf (Already exists)
File 11987/103742: Skipped A0160031.pdf (Already exists)
File 11988/103742: Skipped A0160033.pdf (Already exists)
File 11989/103742: Skipped A0160034.pdf (Already exists)
File 11990/103742: Skipped A0160037.pdf (Already exists)
File 11991/103742: Skipped A0160039.pdf (Already exists)
File 11992/103742: Skipped A0160042.pdf (Already exists)
File 11993/103742: Skipped A0160044.pdf (Already exists)
File 11994/103742: Skipped A016

File 15453/103742: Skipped A0173714.pdf (Already exists)
File 15454/103742: Skipped A0173715.PDF (Already exists)
File 15455/103742: Skipped A0173726.pdf (Already exists)
File 15456/103742: Skipped A0173728.pdf (Already exists)
File 15457/103742: Skipped A0173729_MedRec1.pdf (Duplicate patient file)
File 15458/103742: Skipped A0173730.pdf (Already exists)
File 15459/103742: Skipped A0173734.pdf (Already exists)
File 15460/103742: Skipped A0173736.pdf (Already exists)
File 15461/103742: Skipped A0173743.pdf (Already exists)
File 15462/103742: Skipped A0173754.pdf (Already exists)
File 15463/103742: Skipped A0173756.pdf (Already exists)
File 15464/103742: Skipped A0173766.pdf (Already exists)
File 15465/103742: Skipped A0173769.pdf (Already exists)
File 15466/103742: Skipped A0173771.pdf (Already exists)
File 15467/103742: Skipped A0173779_MedRec1.pdf (Duplicate patient file)
File 15468/103742: Skipped A0173781.pdf (Already exists)
File 15469/103742: Skipped A0173782.pdf (Already exists)

...


sent 308701 bytes  received 42 bytes  205828.67 bytes/sec
total size is 308539  speedup is 1.00
File 93370/103742: Successfully copied A46699.pdf (Extraction Time: 0.34 seconds)
building file list ... done
A46700.pdf

sent 1863662 bytes  received 42 bytes  3727408.00 bytes/sec
total size is 1863312  speedup is 1.00
File 93371/103742: Successfully copied A46700.pdf (Extraction Time: 0.95 seconds)
building file list ... done
A46702.pdf

sent 1293472 bytes  received 42 bytes  862342.67 bytes/sec
total size is 1293190  speedup is 1.00
File 93372/103742: Successfully copied A46702.pdf (Extraction Time: 0.92 seconds)
building file list ... done
A46703.pdf

sent 291772 bytes  received 42 bytes  194542.67 bytes/sec
total size is 291614  speedup is 1.00
File 93373/103742: Successfully copied A46703.pdf (Extraction Time: 0.37 seconds)
building file list ... done
A46705.pdf

sent 6876023 bytes  received 42 bytes  1250193.64 bytes/sec
total size is 6875061  speedup is 1.00
File 93374/103742: Succ


sent 631479 bytes  received 42 bytes  1263042.00 bytes/sec
total size is 631277  speedup is 1.00
File 93408/103742: Successfully copied A46763.pdf (Extraction Time: 0.53 seconds)
building file list ... done
A46764.pdf

sent 319874 bytes  received 42 bytes  213277.33 bytes/sec
total size is 319712  speedup is 1.00
File 93409/103742: Successfully copied A46764.pdf (Extraction Time: 0.69 seconds)
building file list ... done
A46765.pdf

sent 440673 bytes  received 42 bytes  881430.00 bytes/sec
total size is 440495  speedup is 1.00
File 93410/103742: Successfully copied A46765.pdf (Extraction Time: 0.57 seconds)
building file list ... done
A46767.pdf

sent 841958 bytes  received 42 bytes  561333.33 bytes/sec
total size is 841732  speedup is 1.00
File 93411/103742: Successfully copied A46767.pdf (Extraction Time: 0.86 seconds)
building file list ... done
A46768.pdf

sent 2514578 bytes  received 42 bytes  1005848.00 bytes/sec
total size is 2514148  speedup is 1.00
File 93412/103742: Successf

A46864.pdf

sent 268524 bytes  received 42 bytes  537132.00 bytes/sec
total size is 268366  speedup is 1.00
File 93446/103742: Successfully copied A46864.pdf (Extraction Time: 0.40 seconds)
building file list ... done
A46865.pdf

sent 723713 bytes  received 42 bytes  482503.33 bytes/sec
total size is 723499  speedup is 1.00
File 93447/103742: Successfully copied A46865.pdf (Extraction Time: 0.56 seconds)
building file list ... done
A46868.pdf

sent 1185793 bytes  received 42 bytes  790556.67 bytes/sec
total size is 1185523  speedup is 1.00
File 93448/103742: Successfully copied A46868.pdf (Extraction Time: 0.60 seconds)
building file list ... done
A46869.pdf

sent 1345898 bytes  received 42 bytes  2691880.00 bytes/sec
total size is 1345608  speedup is 1.00
File 93449/103742: Successfully copied A46869.pdf (Extraction Time: 0.64 seconds)
building file list ... done
A46869_2.pdf

sent 1025395 bytes  received 42 bytes  683624.67 bytes/sec
total size is 1025143  speedup is 1.00
File 93450/

rsync: writefd_unbuffered failed to write 217 bytes [generator]: Broken pipe (32)


KeyboardInterrupt: 